# Lecture 12: Model evaluation and comparison (3) & GLM

## Instructor： 胡传鹏（博士）[Dr. Hu Chuan-Peng]

### 南京师范大学心理学院[School of Psychology, Nanjing Normal University]

##  Recap: Model evaluation & comparison (3)

### 模型评估/比较与选择(Model evaluation/comparison & selection)

- 过拟合

- 欠拟合



### 常用的模型评估与比较的统计指标可被分为三类。 

- 模型的拟合优度(Goodness of fit)

-- 拟合优度(Goodness of fit)

-- 均方差(Mean squared error, MSE)

-- 对数似然(Log-likelihood)




* 对新数据的预测准确性
	* 交叉验证

* 信息准则(information criteria)
	* AIC (Akaike information criterion)
	* DIC (Deviance information criterion)
	* WAIC (Widely applicable information criterion)
	* LOO-CV (Leave-one-out cross-validation)

* 模型平均(model averaging)


## Example of Model Comparison

![Image Name](https://cdn.kesci.com/upload/image/rkz1ehen1l.png?imageView2/0/w/960/h/960)

### (1) 提出研究问题

Stroop任务是心理学中最常见的认知任务之一，可用于反应抑制能力。

![](https://www.researchgate.net/profile/Ata_Akin/publication/281167153/figure/download/fig1/AS:391418049777669@1470332743733/Three-different-stimulus-conditions-in-the-Stroop-task-neutral-congruent-and.png?_sg=ibeklp8QZ2sbyR29ZZxbOgfS--_RjcKP_uVY36qBahzEJlnMLYPxQyzgYT2Au85eDBClhLqol0A)

个体对于不一致(incongruent)刺激条件下的表现比一致(incongruent)条件下更差，表现为：（1）反应时间往往更长；（2）正确率往往更低。

如果估计不同条件下的个体在反应时间上表现的差异？

图片来源见[这里](https://www.researchgate.net/publication/281167153_Similarity_analysis_of_functional_connectivity_with_functional_near-infrared_spectroscopy/figures?lo=1&utm_source=bing&utm_medium=organic)。

### (2) 查看原始数据

In [6]:
import pandas as pd
import numpy as np
import arviz as az
import pymc3 as pm
import matplotlib.pyplot as plt

# theano.tensor是一个对向量进行操作的模块
import theano.tensor as tt

# random是一个用于进行随机数操作的包
import random

In [7]:
raw_data = pd.read_csv('/home/mw/input/data4676/stroop.csv') # 载入数据
raw_data.head(5) # 查看前5行数据

,Unnamed: 0,battery_name,condition,correct,correct_response,exp_stage,experiment_exp_id,finishtime,focus_shifts,full_screen,key_press,possible_responses,rt,stim_color,stim_word,stimulus,time_elapsed,trial_id,trial_type,worker_id
0,stroop_s000_004,Self Regulation Battery,incongruent,0.0,66.0,practice,stroop,2016-07-22 04:44:22,0,True,71.0,"[66, 71, 82]",1141,blue,red,<div class = centerbox><div class = stroop-sti...,24318,stim,poldrack-categorize,s001
1,stroop_s000_006,Self Regulation Battery,congruent,1.0,82.0,practice,stroop,2016-07-22 04:44:22,0,True,82.0,"[66, 71, 82]",1048,red,red,<div class = centerbox><div class = stroop-sti...,27574,stim,poldrack-categorize,s001
2,stroop_s000_008,Self Regulation Battery,congruent,1.0,66.0,practice,stroop,2016-07-22 04:44:22,0,True,66.0,"[66, 71, 82]",855,blue,blue,<div class = centerbox><div class = stroop-sti...,30828,stim,poldrack-categorize,s001
3,stroop_s000_010,Self Regulation Battery,congruent,1.0,71.0,practice,stroop,2016-07-22 04:44:22,0,True,71.0,"[66, 71, 82]",421,green,green,<div class = centerbox><div class = stroop-sti...,34083,stim,poldrack-categorize,s001
4,stroop_s000_012,Self Regulation Battery,congruent,1.0,71.0,practice,stroop,2016-07-22 04:44:22,0,True,71.0,"[66, 71, 82]",694,green,green,<div class = centerbox><div class = stroop-sti...,37340,stim,poldrack-categorize,s001


In [8]:
random.seed(123) # 设置随机数种子，使每次随机数取样的结果一致
samps = random.sample(list(raw_data['worker_id'].unique()), 1) #随机选取1个被试, unique对于一维数组或者列表，unique: 函数去除其中重复的元素，并按元素由大到小返回一个新的无元素重复的元组或者列表
data = raw_data[(raw_data.worker_id.isin(samps)) & (raw_data.exp_stage == "test") & (raw_data.rt > 0)] # 选取数据中随机选取到的10名被试在测试阶段的反应时大于0的数据
# isin:单一条件筛选
data = data[["worker_id","correct","condition", "rt"]] # 选取数据中的判断正确率，刺激条件，和被试编号
data.reset_index(inplace=True, drop=True) # 重置每个试次的编号
data.head(5) # 查看数据的前5行

,worker_id,correct,condition,rt
0,s058,1.0,incongruent,480
1,s058,1.0,congruent,576
2,s058,1.0,incongruent,757
3,s058,1.0,congruent,760
4,s058,1.0,congruent,640


In [9]:
random.seed(123) # 设置随机数种子，使每次随机数取样的结果一致
samps = random.sample(list(raw_data['worker_id'].unique()), 1) #随机选取1个被试, unique对于一维数组或者列表，unique: 函数去除其中重复的元素，并按元素由大到小返回一个新的无元素重复的元组或者列表
data = raw_data[(raw_data.worker_id.isin(samps)) & (raw_data.exp_stage == "test") & (raw_data.rt > 0)] # 选取数据中随机选取到的10名被试在测试阶段的反应时大于0的数据
data


,Unnamed: 0,battery_name,condition,correct,correct_response,exp_stage,experiment_exp_id,finishtime,focus_shifts,full_screen,key_press,possible_responses,rt,stim_color,stim_word,stimulus,time_elapsed,trial_id,trial_type,worker_id
6384,stroop_s053_053,Self Regulation Battery,incongruent,1.0,82.0,test,stroop,2016-07-28 17:39:28,0,True,82.0,"[66, 71, 82]",480,red,blue,<div class = centerbox><div class = stroop-sti...,108522,stim,poldrack-categorize,s058
6385,stroop_s053_055,Self Regulation Battery,congruent,1.0,82.0,test,stroop,2016-07-28 17:39:28,0,True,82.0,"[66, 71, 82]",576,red,red,<div class = centerbox><div class = stroop-sti...,111801,stim,poldrack-categorize,s058
6386,stroop_s053_057,Self Regulation Battery,incongruent,1.0,82.0,test,stroop,2016-07-28 17:39:28,0,True,82.0,"[66, 71, 82]",757,red,blue,<div class = centerbox><div class = stroop-sti...,115082,stim,poldrack-categorize,s058
6387,stroop_s053_059,Self Regulation Battery,congruent,1.0,82.0,test,stroop,2016-07-28 17:39:28,0,True,82.0,"[66, 71, 82]",760,red,red,<div class = centerbox><div class = stroop-sti...,118365,stim,poldrack-categorize,s058
6388,stroop_s053_061,Self Regulation Battery,congruent,1.0,71.0,test,stroop,2016-07-28 17:39:28,0,True,71.0,"[66, 71, 82]",640,green,green,<div class = centerbox><div class = stroop-sti...,121647,stim,poldrack-categorize,s058
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6475,stroop_s053_235,Self Regulation Battery,incongruent,1.0,66.0,test,stroop,2016-07-28 17:39:28,0,True,66.0,"[66, 71, 82]",726,blue,green,<div class = centerbox><div class = stroop-sti...,406922,stim,poldrack-categorize,s058
6476,stroop_s053_237,Self Regulation Battery,congruent,1.0,66.0,test,stroop,2016-07-28 17:39:28,0,True,66.0,"[66, 71, 82]",563,blue,blue,<div class = centerbox><div class = stroop-sti...,410195,stim,poldrack-categorize,s058
6477,stroop_s053_239,Self Regulation Battery,congruent,1.0,82.0,test,stroop,2016-07-28 17:39:28,0,True,82.0,"[66, 71, 82]",546,red,red,<div class = centerbox><div class = stroop-sti...,413482,stim,poldrack-categorize,s058
6478,stroop_s053_241,Self Regulation Battery,congruent,1.0,82.0,test,stroop,2016-07-28 17:39:28,0,True,82.0,"[66, 71, 82]",497,red,red,<div class = centerbox><div class = stroop-sti...,416762,stim,poldrack-categorize,s058


In [10]:
def filter_ab(df):
    '''
    判断数据是否在平均值三个标准差以内
    '''
    mp1 = df.mean() - 3 * df.std()
    mp2 = df.mean() + 3 * df.std()
    norm = df.where((df > mp1) & (df < mp2)) # 判断数据是否在三个标准差以内，不在则赋值为NA
    return norm
data.rt = data.groupby('worker_id').rt.apply(filter_ab) # 将数据按照被试进行分组，分别检查每个被试的反应时是否在三个标准差以内
data = data.dropna(axis=0, how='any') # 将含NAN的行进行删除
data.reset_index(inplace=True, drop=True) # 重置每个试次的编号


/opt/conda/lib/python3.7/site-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [11]:
# 按照条件进行分组，绘制反应时间的概率密度图
data.groupby('condition').rt.describe()

,count,mean,std,min,25%,50%,75%,max
condition,,,,,,,,
congruent,48.0,602.145833,151.296664,367.0,509.00,552.0,646.25,1144.0
incongruent,48.0,726.729167,151.551794,480.0,608.25,718.0,847.00,1066.0


In [12]:
# 按照条件进行分组，绘制反应时间的概率密度图
data.groupby(['condition']).rt.plot.density() 

condition
congruent      AxesSubplot(0.125,0.125;0.775x0.755)
incongruent    AxesSubplot(0.125,0.125;0.775x0.755)
Name: rt, dtype: object

<Figure size 432x288 with 1 Axes>

In [13]:
# 按照条件进行分组，绘制反应时间的直方图
data.groupby('condition').rt.plot.hist(bins=30, alpha=0.5)

condition
congruent      AxesSubplot(0.125,0.125;0.775x0.755)
incongruent    AxesSubplot(0.125,0.125;0.775x0.755)
Name: rt, dtype: object

<Figure size 432x288 with 1 Axes>

In [14]:
# 将‘condition’条件进行0， 1编码，congruent转化为1，incongruent转化为0，以便在pymc3中进行计算（虚拟编码，线性模型包含了ANOVA，离散变量的线性回归）
data.condition = data.condition.map({'congruent':1, 'incongruent':0})

从观察数据中获得的信息：
* 反应时间数据是“连续”的数据；
* 反应时间的分布看起来不太像是正常分布；
* 两种条件下的差异比较微妙。
* 不同被试的反应时分布存在差异

可能的模型：
* 常规做法，以高低冲突作为自变量，反应时间作为因变量，采用线性模型（即正态分布作为likelihood），检验高低冲突对反应时总体的均值的影响；
* 广义线性模型，尝试非正态分布作为likelihood：log-normal，gamma, ex-Gausian.

通过贝叶斯分析来比较来推断高低冲突对反应时间的影响，其中，需要进行模型比较来选择最优的模型。

#### Model 1: Normal

首先，我们假定反应时是呈正态分布的，正态分布的均值是由高一致性条件和低一致性条件决定的。


![Image Name](https://docs.pymc.io/en/v3/_images/continuous-18.png)

图片来源：https://docs.pymc.io/en/v3/api/distributions/continuous.html#pymc3.distributions.continuous.Normal

##### 模型建构与先验选择？？？？？？？？？？？？？？？？？？？？？？？？？？？？？？？？？？？？？？？？？？？？？？？？？？？？？？？？？？

写作线性模型的概率形式：

$y_i \sim N(\mu, \sigma)$----------y是反应时的预测值

$\mu = \alpha + \beta * cond_{i}$

$\alpha \sim N(700, 200)$

$\beta \sim N(100, 200)$

$\sigma \sim HN(100)$

In [15]:
with pm.Model() as NormalModel:
    # 先验分布: alpha, beta, sigma这三个参数是随机变量
    sigma = pm.HalfNormal('sigma', sd=200)
    alpha = pm.Normal('alpha', mu=700, sd=200)
    beta = pm.Normal('beta', mu=100, sd=200)
    # 自变量condition是之前已经载入的数据
    x = pm.Data("x", data['condition'])
    # 正态分布均值是确定性随机变量，这个变量的值完全由右端值确定
    mu = pm.Deterministic("mu", alpha + beta*x) 
    # Y的观测值，这是一个特殊的观测随机变量，表示模型数据的可能性。也可以表示模型的似然，通过 observed 参数来告诉这个变量其值是已经被观测到了的，不会被拟合算法改变
    # 假定因变量服从正态分布
    y_obs = pm.Normal('y_obs', mu=mu, sd=sigma, observed=data['rt'] )

In [16]:
# 展示模型结构
pm.model_to_graphviz(NormalModel)

##### 先验检验(prior predictive check)

先验的设定是否合理？

可以通过先验预测检验（ Prior Predictive Distribution ）来进行初步的判断。

先验预测检验：利用模型和先验生成假数据并利用这些生成的假数据来评估先验是否合理的过程。

In [17]:
with NormalModel:
    # 先验预测检查
    prior_checks = pm.sample_prior_predictive(samples=50)

In [19]:
x = np.random.randint(2, size = 50) #生成50个假数据，取值为[0,1]

for a, b in zip(prior_checks["alpha"], prior_checks["beta"]):
    y = a + b * x 
    plt.plot(x, y)

# x=0，1时候,预测y=rt的范围是100-1300，可以接受

<Figure size 432x288 with 1 Axes>

通过先验预测检查，我们发现一致和不一致条件的因变量的取值分别在200-1200, 0-1400之间，还算合理。

##### 计算后验

In [20]:
with NormalModel:
    # 不直接使用InferenceData的理由是为了便于之后的模型比较
    trace1_for_comp = pm.sample(draws = 2000, tune=1000, target_accept=0.9,chains=2, cores= 2)    
    # 将pymc的采样对象转化为inferencedata
    trace1=az.from_pymc3(trace1_for_comp)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  This is separate from the ipykernel package so we can avoid doing imports until
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [beta, alpha, sigma]


Sampling 2 chains for 1_000 tune and 2_000 draw iterations (2_000 + 4_000 draws total) took 5 seconds.


##### MCMC诊断

计算后验分布之后，我们需要检查MCMC的采样情况是否稳定，我们一般采取目视检查法和统计数据$\hat{R}$。

In [21]:
# 绘制各参数的采样情况
az.plot_trace(trace1,var_names=['alpha','beta','sigma'])

array([[<AxesSubplot:title={'center':'alpha'}>,
        <AxesSubplot:title={'center':'alpha'}>],
       [<AxesSubplot:title={'center':'beta'}>,
        <AxesSubplot:title={'center':'beta'}>],
       [<AxesSubplot:title={'center':'sigma'}>,
        <AxesSubplot:title={'center':'sigma'}>]], dtype=object)

<Figure size 864x432 with 6 Axes>

In [15]:
az.summary(trace1, var_names=["alpha", "beta", "sigma"])

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
alpha,723.592,22.477,682.228,767.571,0.535,0.379,1766.0,2211.0,1.0
beta,-118.837,31.347,-176.663,-60.538,0.775,0.548,1640.0,2539.0,1.0
sigma,152.954,11.371,132.294,173.876,0.236,0.167,2303.0,2439.0,1.0


根据采样得到的trace比较稳定，并且r_hat取值接近1说明MCMC采样结果稳定。

##### 模型评估(PPC)

在该模型的采样情况达到稳定之后，我们需要判断该模型能够预测真实数据，这就需要使用后验预测分布进行检查。

In [22]:
with NormalModel:
     # pm.sample_posterior_predictive()利用trace.posterior的后验分布计算后验预测分布
    ppc_y = pm.sample_posterior_predictive(trace1.posterior) 
#将ppc_y转化为InferenceData对象合并到trace中
az.concat(trace1, az.from_pymc3(posterior_predictive=ppc_y), inplace=True)

/opt/conda/lib/python3.7/site-packages/arviz/data/io_pymc3.py:100: FutureWarning: Using `from_pymc3` without the model will be deprecated in a future release. Not using the model will return less accurate and less useful results. Make sure you use the model argument or call from_pymc3 within a model context.
  FutureWarning,


In [23]:
# 绘制后验预测分布
az.plot_ppc(trace1)

<AxesSubplot:xlabel='y_obs'>

/opt/conda/lib/python3.7/site-packages/IPython/core/pylabtools.py:151: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  fig.canvas.print_figure(bytes_io, **kw)


<Figure size 432x288 with 1 Axes>

#### Model2: Log-normal

然后，我们假定反应时是呈log正态分布的，LogNormal分布的均值是由高一致性条件和低一致性条件决定的。

![Image Name](https://docs.pymc.io/en/v3/_images/continuous-16.png)


图片来源：https://docs.pymc.io/en/v3/api/distributions/continuous.html#pymc3.distributions.continuous.LogitNormal

##### 模型建构与先验选择

写作线性模型的概率形式：

$y_i \sim lognormal(\mu, \sigma)$   # 将normal 换成lognormal

$\mu = \alpha + \beta * cond_{i}$

$\alpha \sim N(700, 200)$

$\beta \sim N(100, 200)$

$\sigma \sim HN(100)$

In [24]:
with pm.Model() as LogNormal:
    # 先验分布: alpha, beta, sigma这三个参数是随机变量
    sigma = pm.HalfNormal('sigma', sd=100)
    alpha = pm.Normal('alpha', mu=700, sd=200)
    beta = pm.Normal('beta', mu=100, sd=200)
    # 自变量conf是之前已经载入的数据
    x = pm.Data("x", data['condition'])
    # 正态分布均值是确定性随机变量，这个变量的值完全由右端值确定
    mu = pm.Deterministic("mu", alpha + beta*x) 
    # Y的观测值，这是一个特殊的观测随机变量，表示模型数据的可能性。也可以表示模型的似然，通过 observed 参数来告诉这个变量其值是已经被观测到了的，不会被拟合算法改变
    # 假定因变量服从log-normal分布
    y_obs = pm.Lognormal('y_obs',mu=mu,sd=sigma,observed=data['rt'] )

In [25]:
# 展示模型结构
pm.model_to_graphviz(LogNormal)

##### 先验检验(prior predictive check)

先验的设定是否合理？

可以通过先验预测检验（ Prior Predictive Distribution ）来进行初步的判断。

先验预测检验：利用模型和先验生成假数据并利用这些生成的假数据来评估先验是否合理的过程。

In [26]:
with LogNormal:
    # 先验预测检查
    prior_checks = pm.sample_prior_predictive(samples=50)

/opt/conda/lib/python3.7/site-packages/pymc3/distributions/continuous.py:1864: RuntimeWarning: overflow encountered in exp
  return np.exp(mu + (tau ** -0.5) * samples)


In [27]:
x = np.random.randint(2, size = 50) #生成50个假数据，取值为[0,1]

for a, b in zip(prior_checks["alpha"], prior_checks["beta"]):
    y = a + b * x 
    plt.plot(x, y)

<Figure size 432x288 with 1 Axes>

通过先验预测检查，我们发现一致和不一致条件的因变量的取值分别在400-1000，200-1400之间，比较合理。

##### 计算后验

In [28]:
with LogNormal:
    # 不直接使用InferenceData的理由是为了便于之后的模型比较
    trace2_for_comp = pm.sample(draws = 2000, tune=1000, target_accept=0.9,chains=2, cores= 2)    
    # 将pymc的采样对象转化为inferencedata????????????????????????????????????????????????????????????
    trace2=az.from_pymc3(trace2_for_comp)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  This is separate from the ipykernel package so we can avoid doing imports until
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [beta, alpha, sigma]


Sampling 2 chains for 1_000 tune and 2_000 draw iterations (2_000 + 4_000 draws total) took 7 seconds.


##### MCMC诊断

计算后验分布之后，我们需要检查MCMC的采样情况是否稳定，我们一般采取目视检查法和统计数据r_hat

In [29]:
# 绘制各参数的采样情况
az.plot_trace(trace1,var_names=['alpha','beta','sigma'])

array([[<AxesSubplot:title={'center':'alpha'}>,
        <AxesSubplot:title={'center':'alpha'}>],
       [<AxesSubplot:title={'center':'beta'}>,
        <AxesSubplot:title={'center':'beta'}>],
       [<AxesSubplot:title={'center':'sigma'}>,
        <AxesSubplot:title={'center':'sigma'}>]], dtype=object)

<Figure size 864x432 with 6 Axes>

In [24]:
az.summary(trace2, var_names=["alpha", "beta", "sigma"])

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
alpha,6.567,0.032,6.505,6.625,0.001,0.001,1765.0,2183.0,1.0
beta,-0.193,0.046,-0.277,-0.106,0.001,0.001,1901.0,2097.0,1.0
sigma,0.219,0.017,0.189,0.251,0.000,0.000,2214.0,2490.0,1.0


根据采样得到的trace比较稳定，并且$\hat{R}$取值接近1说明MCMC采样结果稳定。

##### 模型评估(PPC)

在该模型的采样情况达到稳定之后，我们需要判断该模型能够预测真实数据，这就需要使用后验预测分布进行检查。

In [31]:
with LogNormal:
     # pm.sample_posterior_predictive()利用trace.posterior的后验分布计算后验预测分布
    ppc_y = pm.sample_posterior_predictive(trace2.posterior) 
#将ppc_y转化为InferenceData对象合并到trace中
az.concat(trace2, az.from_pymc3(posterior_predictive=ppc_y), inplace=True)

/opt/conda/lib/python3.7/site-packages/arviz/data/io_pymc3.py:100: FutureWarning: Using `from_pymc3` without the model will be deprecated in a future release. Not using the model will return less accurate and less useful results. Make sure you use the model argument or call from_pymc3 within a model context.
  FutureWarning,


In [32]:
# 绘制后验预测分布
az.plot_ppc(trace2)

<AxesSubplot:xlabel='y_obs'>

<Figure size 432x288 with 1 Axes>

#### Model3: Gamma

我们假定反应时是呈Gamma分布。

Gamma分布有两个参数$\alpha$, $\beta$，类似于正态分布中的的$\mu$, $\sigma$。

检验$\alpha$是否受到冲突水平的影响。


![Image Name](https://docs.pymc.io/en/v3/_images/continuous-6.png)

图片来源：https://docs.pymc.io/en/v3/api/distributions/continuous.html#pymc3.distributions.continuous.Gamma

##### 模型建构与先验选择

写作线性模型的概率形式：

$y_i \sim gamma(\mu, \sigma)$   # 将normal 换成gamma

$\mu = \alpha + \beta * cond_{i}$

$\alpha \sim N(700, 200)$

$\beta \sim N(100, 200)$

$\sigma \sim HN(100)$


In [33]:
with pm.Model() as Gamma:
    # 先验分布: alpha, beta, sigma这三个参数是随机变量
    sigma = pm.HalfNormal('sigma', sd=100)
    alpha = pm.Normal('alpha', mu=700, sd=200)
    beta = pm.Normal('beta', mu=100, sd=200)
    # 自变量conf是之前已经载入的数据
    x = pm.Data("x", data['condition'])
    # 正态分布均值是确定性随机变量，这个变量的值完全由右端值确定
    mu = pm.Deterministic("mu", alpha + beta*x) 
    # Y的观测值，这是一个特殊的观测随机变量，表示模型数据的可能性。也可以表示模型的似然，通过 observed 参数来告诉这个变量其值是已经被观测到了的，不会被拟合算法改变
    # 假定因变量服从log-normal分布
    y_obs = pm.Gamma('y_obs',mu=mu,sd=sigma,observed=data['rt'] )

In [34]:
# 展示模型结构
pm.model_to_graphviz(Gamma)

##### 先验检验(prior predictive check)

先验的设定是否合理？

可以通过先验预测检验（ Prior Predictive Distribution ）来进行初步的判断。

先验预测检验：利用模型和先验生成假数据并利用这些生成的假数据来评估先验是否合理的过程。

In [35]:
with Gamma:
    # 先验预测检查
    prior_checks = pm.sample_prior_predictive(samples=50)

In [36]:
x = np.random.randint(2, size = 50) #生成50个假数据，取值为[0,1]

for a, b in zip(prior_checks["alpha"], prior_checks["beta"]):
    y = a + b * x 
    plt.plot(x, y)

<Figure size 432x288 with 1 Axes>

通过先验预测检查，我们发现一致和不一致条件的因变量的取值分别在< 200 ms ~ 1200 ms，400 - 1300之间，比较合理

##### 计算后验

In [37]:
with Gamma:
    # 不直接使用InferenceData的理由是为了便于之后的模型比较
    trace3_for_comp = pm.sample(draws = 2000, tune=1000, target_accept=0.9,chains=2, cores= 2)    
    # 将pymc的采样对象转化为inferencedata
    trace3 = az.from_pymc3(trace3_for_comp)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  This is separate from the ipykernel package so we can avoid doing imports until
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [beta, alpha, sigma]


Sampling 2 chains for 1_000 tune and 2_000 draw iterations (2_000 + 4_000 draws total) took 5 seconds.


##### MCMC诊断

计算后验分布之后，我们需要检查MCMC的采样情况是否稳定，我们一般采取目视检查法和统计数据r_hat

In [38]:
# 绘制各参数的采样情况
az.plot_trace(trace3,var_names=['alpha','beta','sigma'])

array([[<AxesSubplot:title={'center':'alpha'}>,
        <AxesSubplot:title={'center':'alpha'}>],
       [<AxesSubplot:title={'center':'beta'}>,
        <AxesSubplot:title={'center':'beta'}>],
       [<AxesSubplot:title={'center':'sigma'}>,
        <AxesSubplot:title={'center':'sigma'}>]], dtype=object)

<Figure size 864x432 with 6 Axes>

In [39]:
az.summary(trace3, var_names=["alpha", "beta", "sigma"])

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
alpha,722.325,20.671,684.325,761.712,0.510,0.361,1649.0,2111.0,1.0
beta,-114.275,28.656,-170.080,-60.706,0.674,0.476,1808.0,2107.0,1.0
sigma,145.662,11.289,125.257,166.568,0.234,0.167,2379.0,2247.0,1.0


根据采样得到的trace比较稳定，并且r_hat取值接近1说明MCMC采样结果稳定。

##### 模型评估(PPC)

在该模型的采样情况达到稳定之后，我们需要判断该模型能够预测真实数据，这就需要使用后验预测分布进行检查。

In [40]:
with Gamma:
     # pm.sample_posterior_predictive()利用trace.posterior的后验分布计算后验预测分布
    ppc_y = pm.sample_posterior_predictive(trace3.posterior) 
#将ppc_y转化为InferenceData对象合并到trace中
az.concat(trace3, az.from_pymc3(posterior_predictive=ppc_y), inplace=True)

/opt/conda/lib/python3.7/site-packages/arviz/data/io_pymc3.py:100: FutureWarning: Using `from_pymc3` without the model will be deprecated in a future release. Not using the model will return less accurate and less useful results. Make sure you use the model argument or call from_pymc3 within a model context.
  FutureWarning,


In [41]:
# 绘制后验预测分布
az.plot_ppc(trace3)

<AxesSubplot:xlabel='y_obs'>

<Figure size 432x288 with 1 Axes>

#### Model4: ex-Gaussian

我们假定反应时是ex-Gaussian 模型。

该模型有三个参数：$\mu$, $\sigma$, $\tau$

![Image Name](https://docs.pymc.io/en/latest/_images/pymc-ExGaussian-1.png)

图片来源：https://docs.pymc.io/en/latest/api/distributions/generated/pymc.ExGaussian.html

##### 模型建构与先验选择

写作线性模型的概率形式：

$y_i \sim exgaussian(\mu, \sigma, \tau)$   # 将normal 换成 exgaussian

$\mu = \alpha + \beta * cond_{i}$

$\alpha \sim N(700,200)$

$\beta \sim N(100,200)$

$\sigma \sim HN(100)$

$\tau \sim HN(100)$

In [43]:
with pm.Model() as exgaussian:
    # 先验分布:alpha,beta,sigma,nu这三个参数是随机变量
    alpha = pm.Normal('alpha', mu = 700, sd=200)
    beta = pm.Normal('beta',mu=100, sd=200)
    sigma = pm.HalfNormal('sigma', sd=200) 
    nu = pm.HalfNormal('nu', sd=100)   
    # 自变量conf是之前已经载入的数据
    x = pm.Data("x", data['condition'])
    # 参数mu是确定性随机变量，这个变量的值完全由右端值确定
    mu = pm.Deterministic("mu",  alpha+ beta*x) 

    # Y的观测值，这是一个特殊的观测随机变量，表示模型数据的可能性。也可以表示模型的似然，通过 observed 参数来告诉这个变量其值是已经被观测到了的，不会被拟合算法改变
    y_obs = pm.ExGaussian('y_obs',mu=mu,sigma=sigma,nu=nu,observed=data['rt'] )

In [44]:
pm.model_to_graphviz(exgaussian)

##### 先验检验(prior predictive check)

先验的设定是否合理？

可以通过先验预测检验（ Prior Predictive Distribution ）来进行初步的判断。

先验预测检验：利用模型和先验生成假数据并利用这些生成的假数据来评估先验是否合理的过程。

In [45]:
with exgaussian:
    
    # 先验预测检查
    prior_checks = pm.sample_prior_predictive(samples=50)

In [46]:
x = np.random.randint(2, size = 50) #生成50个假数据，取值为[0,1]

for a, b in zip(prior_checks["alpha"], prior_checks["beta"]):
    y = a + b * x 
    plt.plot(x, y)

<Figure size 432x288 with 1 Axes>

##### 计算后验

In [47]:
with exgaussian:
    trace4_for_comp = pm.sample(draws = 2000, tune=1000, target_accept=0.9,chains=2, cores= 2)    
    # 将pymc的采样对象转化为inferencedata
    trace4=az.from_pymc3(trace4_for_comp)

WARNING (theano.tensor.opt): Optimization Warning: The Op erfcx does not provide a C implementation. As well as being potentially slow, this also disables loop fusion.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [nu, sigma, beta, alpha]


Sampling 2 chains for 1_000 tune and 2_000 draw iterations (2_000 + 4_000 draws total) took 11 seconds.


##### MCMC诊断

计算后验分布之后，我们需要检查MCMC的采样情况是否稳定，我们一般采取目视检查法和统计数据r_hat

In [48]:
az.summary(trace4, var_names=["alpha", "beta", "sigma","nu"])

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
alpha,573.095,31.069,517.534,632.603,0.991,0.703,1004.0,1505.0,1.0
beta,-98.432,27.158,-151.106,-50.173,0.760,0.547,1297.0,1386.0,1.0
sigma,70.700,19.739,36.626,107.117,0.590,0.417,1115.0,1934.0,1.0
nu,140.478,25.553,94.991,191.542,0.713,0.504,1277.0,1956.0,1.0


In [49]:
# 绘制各参数的采样情况
az.plot_trace(trace4,var_names=["alpha", "beta", "sigma","nu"])

array([[<AxesSubplot:title={'center':'alpha'}>,
        <AxesSubplot:title={'center':'alpha'}>],
       [<AxesSubplot:title={'center':'beta'}>,
        <AxesSubplot:title={'center':'beta'}>],
       [<AxesSubplot:title={'center':'sigma'}>,
        <AxesSubplot:title={'center':'sigma'}>],
       [<AxesSubplot:title={'center':'nu'}>,
        <AxesSubplot:title={'center':'nu'}>]], dtype=object)

<Figure size 864x576 with 8 Axes>

根据采样得到的trace比较稳定，并且r_hat取值接近1说明MCMC采样结果稳定。

##### 模型评估(PPC)

在该模型的采样情况达到稳定之后，我们需要判断该模型能够预测真实数据，这就需要使用后验预测分布进行检查。

In [50]:
with exgaussian:
    # pm.sample_posterior_predictive()利用trace.posterior的后验分布计算后验预测分布
    ppc_y = pm.sample_posterior_predictive(trace4.posterior) 
#将ppc_y转化为InferenceData对象合并到trace中
az.concat(trace4, az.from_pymc3(posterior_predictive=ppc_y), inplace=True)

/opt/conda/lib/python3.7/site-packages/arviz/data/io_pymc3.py:100: FutureWarning: Using `from_pymc3` without the model will be deprecated in a future release. Not using the model will return less accurate and less useful results. Make sure you use the model argument or call from_pymc3 within a model context.
  FutureWarning,


In [51]:
az.plot_ppc(trace4)

<AxesSubplot:xlabel='y_obs'>

<Figure size 432x288 with 1 Axes>

### 模型比较

#### LOO

In [45]:
# 将四个模型的采样结果放到dictionary中进行比较
compare_dict = {"normal": trace1, "log-normal": trace2, "gamma": trace3,"exgaussian":trace4}
# 选择loo方法进行比较
comp = az.compare(compare_dict, ic='loo')
comp

/opt/conda/lib/python3.7/site-packages/arviz/stats/stats.py:146: UserWarning: The default method used to estimate the weights for each model,has changed from BB-pseudo-BMA to stacking
  "The default method used to estimate the weights for each model,"


,rank,loo,p_loo,d_loo,weight,se,dse,warning,loo_scale
log-normal,0,-612.674051,3.196749,0.000000,6.360410e-01,8.412337,0.000000,False,log
exgaussian,1,-613.321656,4.551529,0.647606,3.639590e-01,7.838731,2.742772,False,log
gamma,2,-614.527448,2.904233,1.853397,0.000000e+00,7.928021,1.221702,False,log
normal,3,-620.535490,3.511112,7.861439,1.287859e-14,8.886006,2.213495,False,log


- 第一列为索引，它列出了传递给 az.compare(.) 的模型名称。

- rank 列：按照预测精度做的排名，值从0依次到模型总数，其中0代表最高精度。

- loo 列：各模型 ELPD 值的列表，总是按照 ELPD 值从最好到最差排序。对数似然的变式，越大越好。

- p_loo 列：惩罚项的值列表，可以将其粗略地视为有效参数数量的估计值（但不要太认真）。此值可能低于具有更多结构的模型（如分层模型）中的实际参数数量，或者高于那些预测能力非常弱或严重错误指定的模型的实际参数数量。

- d_loo 列：每个模型与排名第一的模型之间的 LOO 相对差。因此第一个模型始终取值为0  。

- weight 列：分配给每个模型的权重。权重可以粗略地解释为在指定数据的条件下，是（参与比较的各模型中）该模型的概率。

- se 列：ELPD 的标准误差。

- dse 列：ELPD 相对差的标准误差。 dse 与 se 不一定相同，因为 ELPD 的不确定性在模型之间可能存在相关性。排名第一的模型 dse 值始终为0 。

- warning 列：如果为True，表示这是一个警告，LOO 的近似估计不可靠。

- loo_scale 列：估计值所用的尺度。默认为对数尺度。其他选项还包括：离差值尺度，即对数分值乘以-2，这会颠倒排序，较低的 ELPD 会更好；负对数尺度，即对数分值乘以-1，与离差值尺度一样，值越低越好。


#### WAIC

In [46]:
# 将4个模型的采样结果进行比较
compare_dict = {"normal": trace1, "log-normal": trace2, "gamma": trace3,"exgaussian":trace4}
# 选择loo方法进行比较
comp = az.compare(compare_dict, ic='waic')
comp

/opt/conda/lib/python3.7/site-packages/arviz/stats/stats.py:146: UserWarning: The default method used to estimate the weights for each model,has changed from BB-pseudo-BMA to stacking
  "The default method used to estimate the weights for each model,"
/opt/conda/lib/python3.7/site-packages/arviz/stats/stats.py:1407: UserWarning: For one or more samples the posterior variance of the log predictive densities exceeds 0.4. This could be indication of WAIC starting to fail. 
See http://arxiv.org/abs/1507.04544 for details
  "For one or more samples the posterior variance of the log predictive "
/opt/conda/lib/python3.7/site-packages/arviz/stats/stats.py:1407: UserWarning: For one or more samples the posterior variance of the log predictive densities exceeds 0.4. This could be indication of WAIC starting to fail. 
See http://arxiv.org/abs/1507.04544 for details
  "For one or more samples the posterior variance of the log predictive "
/opt/conda/lib/python3.7/site-packages/arviz/stats/stats.p

,rank,waic,p_waic,d_waic,weight,se,dse,warning,waic_scale
log-normal,0,-612.655561,3.178259,0.000000,6.283489e-01,8.402979,0.000000,True,log
exgaussian,1,-613.222991,4.452864,0.567430,3.716511e-01,7.809461,2.719900,True,log
gamma,2,-614.520046,2.896831,1.864485,0.000000e+00,7.925004,1.218672,True,log
normal,3,-620.530952,3.506573,7.875390,6.772360e-15,8.883640,2.218972,True,log


#### DIC & others

In [47]:
def comp_model(trace, model):
    '''
    trace: 未转化为inferencedata的采样结果
    model: 模型
    '''
    dftrc_m = pm.trace_to_dataframe(trace, include_transformed=True) # 将trace对象转化为dataframe
    trace = az.from_pymc3(trace) # 将trace对象转化为inferencedata对象
    trc_logp = trace['log_likelihood']['y_obs'].to_dataframe().groupby(['chain','draw']).sum().reset_index()['y_obs'] #从模型中提取对数似然
    
    #dic
    mean_deviance = -2 * trc_logp.mean(0) #计算所有参数对数似然的平均值
    deviance_at_mean = -2 * model.logp(dftrc_m.mean(0).to_dict()) #计算参数平均值的对数似然
    dic = 2 * mean_deviance - deviance_at_mean
    
    #bic
    #deviance_at_mle = min(trc_logp) #对数似然最小的值
    #parnum = 3 # 参数数量
    #n = 3988 # 数据的样本数
    #bic = -2 * deviance_at_mle +  2*parnum*np.log(n)
    
    #aic
    #aic = -2 * deviance_at_mle +  2*parnum
    
    return dic
    #,bic,aic

In [48]:
# 计算各模型dic的值
dic2 = comp_model(trace=trace2_for_comp,model=LogNormal)
dic4 = comp_model(trace=trace4_for_comp,model=exgaussian)
dic3 = comp_model(trace=trace3_for_comp,model=Gamma)
dic1 = comp_model(trace=trace1_for_comp,model=NormalModel)
dics = {'gamma':dic3,'log-normal':dic2,'exgaussian':dic4,'normal':dic1}

/opt/conda/lib/python3.7/site-packages/arviz/data/io_pymc3.py:100: FutureWarning: Using `from_pymc3` without the model will be deprecated in a future release. Not using the model will return less accurate and less useful results. Make sure you use the model argument or call from_pymc3 within a model context.
  FutureWarning,
/opt/conda/lib/python3.7/site-packages/arviz/data/io_pymc3.py:100: FutureWarning: Using `from_pymc3` without the model will be deprecated in a future release. Not using the model will return less accurate and less useful results. Make sure you use the model argument or call from_pymc3 within a model context.
  FutureWarning,
/opt/conda/lib/python3.7/site-packages/arviz/data/io_pymc3.py:100: FutureWarning: Using `from_pymc3` without the model will be deprecated in a future release. Not using the model will return less accurate and less useful results. Make sure you use the model argument or call from_pymc3 within a model context.
  FutureWarning,
/opt/conda/lib/pyth

In [49]:
model_comp = pd.DataFrame({ 
                        'rank':az.compare(compare_dict, ic='waic')['rank'],
                        'waic': az.compare(compare_dict, ic='waic')['waic'],
                        'loo': az.compare(compare_dict, ic='loo')['loo'],
                        })
model_comp.loc['log-normal','dic'] =dics['log-normal']
model_comp.loc['normal','dic'] =dics['normal']
model_comp.loc['exgaussian','dic'] =dics['exgaussian']
model_comp.loc['gamma','dic'] =dics['gamma']

/opt/conda/lib/python3.7/site-packages/arviz/stats/stats.py:146: UserWarning: The default method used to estimate the weights for each model,has changed from BB-pseudo-BMA to stacking
  "The default method used to estimate the weights for each model,"
/opt/conda/lib/python3.7/site-packages/arviz/stats/stats.py:1407: UserWarning: For one or more samples the posterior variance of the log predictive densities exceeds 0.4. This could be indication of WAIC starting to fail. 
See http://arxiv.org/abs/1507.04544 for details
  "For one or more samples the posterior variance of the log predictive "
/opt/conda/lib/python3.7/site-packages/arviz/stats/stats.py:1407: UserWarning: For one or more samples the posterior variance of the log predictive densities exceeds 0.4. This could be indication of WAIC starting to fail. 
See http://arxiv.org/abs/1507.04544 for details
  "For one or more samples the posterior variance of the log predictive "
/opt/conda/lib/python3.7/site-packages/arviz/stats/stats.p

In [50]:
model_comp

,rank,waic,loo,dic
log-normal,0,-612.655561,-612.674051,1175.294384
exgaussian,1,-613.222991,-613.321656,1194.861382
gamma,2,-614.520046,-614.527448,1201.247458
normal,3,-620.530952,-620.535490,1212.743009


## Part 2: 广义线性模型(Generalized linear model, GLM)

本节的目的在与：了解如何通过广义线性模型(Generalized linear model, GLM)拟合正确率等二元决策变量。

重点在于：
- 了解使用 Pymc 进行数据分析的完整 workflow
- 了解因变量为正确率等二元决策变量(往往记录为0或1，1代表回答正确，0代表回答错误)的特征
- 了解广义线性模型(Generalized linear model)中的伯努利(Bernoulli)分布和链接函数(link function)


### 什么是GLM？

GLM 是一般线性模型的一种扩展。我们首先介绍一般线性模型的基础知识。

在一般线性模型中，因变量被假定为服从正态分布 $y \sim Normal(\mu,sigma)$。
- 其中，y为观测项；$\mu$为预测项；sigma 为误差项。
- 预测项展开为 $\mu  = \alpha + \beta *x$。
- 例子，比如用员工工龄(x)预测他们的工资(y)，其中x和y都为连续变量。


![Image Name](https://cdn.kesci.com/upload/image/rll49b8jn9.png?imageView2/0/w/640/h/640)

比如，我们这里男性编码为0，将女性编码为1。

$Y = \beta_0 + \beta_1 X + \epsilon$
- 当虚拟变量赋值为X=0时， $E(Y) = \beta_0$ 代表男性的平均家庭收入
- 当虚拟变量赋值为X=1时，$E(Y) = \beta_0+\beta_1$ 代表女性的平均家庭收入
- $\beta_1$ 表示女性相对于男性的家庭收入的差值

- 当自变量为二分变量时，t 检验是线性模型的特殊形式。包括独立样本t检验和配对样本t检验。
- 当自变量为超过俩水平的分类变量时，方差分析是线性模型的特殊形式。包括方差分析和重复测量方差分析。

例子中的性别为二分变量，对应独立样本t检验。

与独立样本t检验不同，在线性模型中检查两组之间的差异相当于检查回归系数 $b_1 $的显著性。

![Image Name](https://cdn.kesci.com/upload/image/rloa41zjxa.png?imageView2/0/w/640/h/640)


**当因变量为离散变量**

比如，因变量为答题正确率，其中1代表回答正确，0代表回答错误。

正确率为离散变量，并不服从正态分布，而是服从伯努利(Bernoulli)分布。


![Image Name](https://cdn.kesci.com/upload/image/rloa62fn8w.png?imageView2/0/w/960/h/960)


![](https://docs.pymc.io/en/v4.3.0/_images/pymc-Bernoulli-1.png)

广义线性模型(Generalized linear model，GLM)的特点：

| 一般线性模型 | 广义线性模型 | 
|---|---|
| $y \sim Normal(\mu,sigma)$ | $y \sim dist(p)$ |
| $\mu = \alpha + \beta *x$ | $p = g(\mu)$|


- 首先，GLM 可以将 $y \sim Normal(\mu,sigma)$ 扩展为 $y \sim Bernoulli(p)$ ，使得因变量y服从伯努利分布。
- 同样，参数 $p$ 可以与自变量联系在一起， $p  = \alpha + \beta * x$。
- 需要注意的是，由于$p$ 的取值范围在[0, 1]，而 $\alpha + \beta * x$ 的范围为 $(-\infty, +\infty)$。我们需要通过**链接函数g()** 将 $\alpha + \beta * x$  映射到 $p$ 所在的范围。




链接函数的具体转化过程，以逻辑(logit)回归为例：
1. 令 $z = \alpha + \beta *x$，$\mu$的范围为 $(-\infty, +\infty)$。
2. $p = g(z)$，其中 g() 为链接函数，输出结果 p 的范围为 $(0,1)$。
3.  最后将 p 输入到分布函数中 $y \sim Bernoulli(p)$。


![Image Name](https://cdn.kesci.com/upload/image/rloa6zyf5a.png?imageView2/0/w/600/h/600)

### Workflow

在了解GLM的基础知识后，我们通过实际的例子来体验PyMC建模的完整workflow。

![Image Name](https://cdn.kesci.com/upload/image/rkvikqg9q6.png?imageView2/0/w/650/h/650)

### (1) 提出研究问题

假如研究问题为：不同刺激条件下**正确率**的差异。

![](https://www.researchgate.net/profile/Ata_Akin/publication/281167153/figure/download/fig1/AS:391418049777669@1470332743733/Three-different-stimulus-conditions-in-the-Stroop-task-neutral-congruent-and.png?_sg=ibeklp8QZ2sbyR29ZZxbOgfS--_RjcKP_uVY36qBahzEJlnMLYPxQyzgYT2Au85eDBClhLqol0A)



图片来源：https://www.researchgate.net/publication/281167153_Similarity_analysis_of_functional_connectivity_with_functional_near-infrared_spectroscopy/figures?lo=1&utm_source=bing&utm_medium=organic

### (2) 数据收集

In [51]:
#加载需要使用的库
%matplotlib inline
import numpy as np 
from scipy import stats
import matplotlib.pyplot as plt
import pandas as pd
import arviz as az
import pymc3 as pm
# 随机数种子，确保随后生成的随机数相同
np.random.seed(123)  

这里我们使用的数据来自 Eisenberg et al (2019)。

为了简化问题，我们仅考虑有一个被试的数据。其中：
- worker_id 为被试编号。
- correct 为被试在 stroop 任务中每个试次判断的正确性，其中1代表判断正确，0代表判断错误。
- condition 为刺激的类别，congruent为颜色和字意一致，incongruent为颜色和字意不一致。

In [52]:
# 加载数据
data = pd.read_csv("/home/mw/input/data4676/stroop.csv")
# 选取第一个被试在正式实验(test)中的数据
data = data[(data.worker_id == "s001") & (data.exp_stage == "test")]
# 选取数据中的判断正确率，刺激条件，和被试编号
data = data[["worker_id","correct","condition"]]
# 重置每个试次的编号
data.reset_index(inplace=True,drop=True)

In [53]:
data.head()

,worker_id,correct,condition
0,s001,1.0,congruent
1,s001,0.0,congruent
2,s001,1.0,congruent
3,s001,1.0,congruent
4,s001,1.0,congruent


In [54]:
data.groupby('condition').correct.describe() 

,count,mean,std,min,25%,50%,75%,max
condition,,,,,,,,
congruent,48.0,0.8750,0.334219,0.0,1.0,1.0,1.0,1.0
incongruent,48.0,0.8125,0.394443,0.0,1.0,1.0,1.0,1.0


In [55]:
data.groupby(['condition']).correct.mean().plot.bar()
plt.show()

<Figure size 432x288 with 1 Axes>

### (3) 选择模型

在我们的例子中，由于因变量(反应的正确性)不是连续变量，因此预测变量不服从正态分布。

二分变量通常采用伯努利(Bernoulli)分布进行分析，因此我们需要广义线性模型(Generalized linear model，GLM)来扩展一般线性模型：
- 首先，GLM 可以将 $y \sim Normal(\mu,sigma)$ 扩展为 **$y \sim Bernoulli(p)$** ，使得因变量y服从伯努利分布。
- 同样，参数 $p$ 可以与自变量联系在一起， $p  = \alpha + \beta * x$。
- 需要注意的是，$p$ 的取值范围在[0, 1]，而 $\alpha + \beta * x$ 的范围为 $(-\infty, +\infty)$。我们需要通过**链接函数** 将 $\alpha + \beta * x$  映射到 $p$ 所在的范围。
	1. 令 $z = \alpha + \beta *x$，$\mu$的范围为 $(-\infty, +\infty)$。
	2. $p = g(z)$，其中 g() 为链接函数，输出结果 $p$ 的范围为 $(0,1)$。
	3.  最后将 $p$ 输入到分布函数中 $y \sim Bernoulli(p)$。

### (4)选择先验

In [56]:
# 将‘condition’进行编码，其中一致条件(congruent)编码为0，不一致条件(incongruent)编码为1。
data.condition = data.condition.map({'incongruent':1,'congruent':0})

In [57]:
# 在pymc3中，pm.Model()定义了一个新的模型对象，这个对象是模型中随机变量的容器
# 在python中，容器是一种数据结构，是用来管理特殊数据的对象
# with语句定义了一个上下文管理器，以 linear_model 作为上下文，在这个上下文中定义的变量都被添加到这个模型
with pm.Model() as GLM_model:
    # 设定先验分布: 
    alpha = pm.Normal('alpha',mu=0,sd=1)
    beta = pm.Normal('beta',mu=0,sd=1)
    # x为自变量，是之前已经载入的数据
    x = pm.Data("x", data['condition'])
    # 通过链接函数对参数进行转换
    z = alpha + beta * x                            # 对应步骤1
    p = pm.Deterministic("p", pm.math.invlogit(z))  # 对应步骤2

    # 先验预测检查
    prior_checks = pm.sample_prior_predictive(samples=50)

In [58]:
az.plot_density(
    {'alpha':prior_checks['alpha'],
    'beta':prior_checks['beta']}
    )
plt.show()

<Figure size 993.6x331.2 with 2 Axes>

In [59]:
az.plot_density(
    {'p':prior_checks['p']}
    )
plt.show()

<Figure size 432x288 with 1 Axes>

结果发现，通过**链接函数**转换后的p值范围为 0到1。

### (5) 拟合数据

首先定义 GLM 模型：
- 其中 alpha 和 beta 为模型参数，$\alpha + \beta * x$。
- x 为自变量刺激条件(condition), 0代表一致条件，1代表不一致条件。
- 通过链接函数对参数进行转换。
    - 首先令 $\mu = \alpha + \beta * x$
    - 然后通过链接函数 pm.math.invlogit(mu)，计算出 $p$。
    注意，这里我们选择 logit 的反函数 invlogit作为链接函数。该链接函数使得 $p$ 的范围为 $(0,1)$。
-  最后将 $p$ 输入到分布函数中 $y \sim Bernoulli(p)$。

In [60]:
with pm.Model() as GLM_model:
    # 在pymc3中，pm.Model()定义了一个新的模型对象，这个对象是模型中随机变量的容器
    # 在python中，容器是一种数据结构，是用来管理特殊数据的对象
    # with语句定义了一个上下文管理器，以 linear_model 作为上下文，在这个上下文中定义的变量都被添加到这个模型
    # 定义先验
    alpha = pm.Normal('alpha', mu=0, sd=1)
    beta = pm.Normal('beta', mu=0, sd=1)
    # x为自变量，是之前已经载入的数据
    x = pm.Data("x", data['condition'])
    # 线性模型：mu是确定性随机变量，这个变量的值完全由右端值确定
    z = alpha + beta * x                            # 对应步骤1
    p = pm.Deterministic("p", pm.math.invlogit(z))  # 对应步骤2
    # Y的观测值，这是一个特殊的观测随机变量，表示模型数据的可能性。也可以表示模型的似然，通过 observed 参数来告诉这个变量其值是已经被观测到了的，不会被拟合算法改变
    y_obs = pm.Bernoulli("y_obs", p=p, observed=data["correct"])  # 对应步骤3

In [61]:
# 展示模型结构
pm.model_to_graphviz(GLM_model)

注意：由于应用 GLM 模型时往往都会使用到链接函数，为了减轻使用者的工作量，在 pymc中可以通过设定 将 `pm.Bernoulli("y_obs", p=p)` 的设定改为 `pm.Bernoulli("y_obs", logit_p=p)` 。 完整代码如下：

In [62]:
with pm.Model() as GLM_model:
    # 定义先验
    alpha = pm.Normal('alpha', mu=0, sd=1)
    beta = pm.Normal('beta', mu=0, sd=1, shape=1)
    # x为自变量，是之前已经载入的数据
    x = pm.Data("x", data['condition'])
    # 线性模型：mu是确定性随机变量，这个变量的值完全由右端值确定
    p = pm.Deterministic("p", alpha + beta * x)
    # Y的观测值，这是一个特殊的观测随机变量，表示模型数据的可能性。也可以表示模型的似然，通过 observed 参数来告诉这个变量其值是已经被观测到了的，不会被拟合算法改变
    y_obs = pm.Bernoulli("y_obs", logit_p=p, observed=data["correct"])

### (6)采样过程诊断

如果使用MCMC对后验进行近似，则需要首先对MCMC过程进行评估。

* 是否收敛；
* 是否接近真实的后验。

对采样过程的评估我们会采用目视检查或rhat这个指标

In [63]:
with GLM_model :
    # 使用mcmc方法进行采样，draws为采样次数，tune为调整采样策略的次数，这些次数将在采样结束后被丢弃，
    # target_accept为接受率， return_inferencedata=True为该函数返回的对象是arviz.InnferenceData对象
    # chains为我们采样的链数，cores为我们的调用的cpu数，多个链可以在多个cpu中并行计算，我们在和鲸中调用的cpu数为2
    trace = pm.sample(draws = 2000, tune=1000, target_accept=0.9,chains=2, cores= 2,return_inferencedata=True)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [beta, alpha]


Sampling 2 chains for 1_000 tune and 2_000 draw iterations (2_000 + 4_000 draws total) took 5 seconds.


In [64]:
az.plot_trace(trace, var_names=['alpha','beta'])
plt.show()

<Figure size 864x288 with 4 Axes>

In [65]:
az.summary(trace, var_names=['alpha','beta'], kind="diagnostics")

,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
alpha,0.010,0.007,1314.0,1578.0,1.0
beta[0],0.013,0.010,1267.0,1500.0,1.0


### (7)模型诊断

在MCMC有效的前提下，需要继续检验模型是否能够较好地拟合数据。

我们会使用后验预测分布通过我们得到的参数生成一批模拟数据，并将其与真实数据进行对比。

In [66]:
# 后验预测分布的计算仍在容器中进行
with GLM_model:
    # pm.sample_posterior_predictive()利用trace.posterior的后验分布计算后验预测分布
    ppc_y = pm.sample_posterior_predictive(trace.posterior) 
#将ppc_y转化为InferenceData对象合并到trace中
az.concat(trace, az.from_pymc3(posterior_predictive=ppc_y), inplace=True)

/opt/conda/lib/python3.7/site-packages/arviz/data/io_pymc3.py:100: FutureWarning: Using `from_pymc3` without the model will be deprecated in a future release. Not using the model will return less accurate and less useful results. Make sure you use the model argument or call from_pymc3 within a model context.
  FutureWarning,


In [67]:
# 绘制后验预测分布
az.plot_ppc(trace)
plt.show()

/opt/conda/lib/python3.7/site-packages/IPython/core/pylabtools.py:151: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  fig.canvas.print_figure(bytes_io, **kw)


<Figure size 432x288 with 1 Axes>

### (8)模型比较

当采样诊断与模型诊断说明模型是否可用后，我们可以通过模型检验来验证我们的研究问题：在不同刺激条件下(一致 vs. 不一致)个体正确率是否存在差异。

![Image Name](https://cdn.kesci.com/upload/image/rkm3pw954u.png?imageView2/0/w/960/h/960)

我们可以定义一个不考虑自变量影响的模型 `GLM_null_model`。如果之前的模型拟合优度好于该模型，那么说明自变量对模型存在影响。

In [68]:
with pm.Model() as GLM_null_model:
    # 定义先验
    p = pm.Uniform('p', 0, 1)  # 由于没有考虑自变量的影响，因此我们可以直接假设参数p服从0到1的均匀分布。
    # Y的观测值，这是一个特殊的观测随机变量，表示模型数据的可能性。也可以表示模型的似然，通过 observed 参数来告诉这个变量其值是已经被观测到了的，不会被拟合算法改变
    y_obs = pm.Bernoulli('y_obs', p=p, observed=data['correct'])

    trace2 = pm.sample(draws = 2000, tune=1000, target_accept=0.9,chains=2, cores= 2,return_inferencedata=True)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [p]


Sampling 2 chains for 1_000 tune and 2_000 draw iterations (2_000 + 4_000 draws total) took 3 seconds.


In [69]:
############################
# 练习
# 要求：完成对 GLM_null_model 的采样过程诊断与模型诊断。
############################

# 绘制各参数的采样情况
# Tips: 使用 az.plot_trace() 函数可以绘制 trace 图；使用 az.summary() 可以得到诊断统计结果


# 模型诊断
# Tips: 使用 pm.sample_posterior_predictive 可以进行后验预测检验；使用 az.plot_ppc() 可以得到后验预测检验图


当对 GLM_null_model 进行同样的检验，我们可以正式进行模型比较了。

In [70]:
# 将三个模型的采样结果进行比较
compare_dict = {"GLM_model": trace, "GLM_null_model": trace2}
# 选择loo方法进行比较
comp = az.compare(compare_dict, ic='loo')
comp

/opt/conda/lib/python3.7/site-packages/arviz/stats/stats.py:146: UserWarning: The default method used to estimate the weights for each model,has changed from BB-pseudo-BMA to stacking
  "The default method used to estimate the weights for each model,"


,rank,loo,p_loo,d_loo,weight,se,dse,warning,loo_scale
GLM_null_model,0,-42.584941,0.954592,0.000000,1.0,5.963029,0.000000,False,log
GLM_model,1,-43.136095,1.660994,0.551154,0.0,5.839979,0.371372,False,log


结果显示，`GLM_null_model` 模型的拟合度好于 `GLM_model`，说明不存在充分的证据表明不同刺激条件会影响个体判断的正确率。

### (9)统计推断

我们可以进一步通过统计推断印证模型比较的结果。

In [71]:
az.plot_posterior(trace, var_names=['beta'])
plt.show()

<Figure size 432x288 with 1 Axes>

参数 $\beta$ 反应了两个条件下正确率的差异。我们可以看到，该参数的后验分布的大部分包括0，说明支持两个条件下正确率存在差异的证据不足。

我们进一步查看两个参数的情况：

In [72]:
az.summary(trace, var_names=['alpha','beta'])

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
alpha,1.706,0.361,1.047,2.374,0.010,0.007,1314.0,1578.0,1.0
beta[0],-0.193,0.477,-1.103,0.716,0.013,0.010,1267.0,1500.0,1.0


结果发现，两个参数值的范围不太“正常”。

还记得 **链接函数**吗？
- 链接函数 g() 将 $\alpha + \beta * x$ 的范围从 $(-\infty, +\infty)$ 转换为 (0,1)
- 同时，其中的参数 $\alpha$ 和 $\beta$也被转换了，只不过他们从 (0,1) 转换为  $(-\infty, +\infty)$，所以  $\alpha$ 大于1，并且 $\beta$小于0。
- 为了他们转换回来，我们需要使用 logit 函数， $p = \frac{1}{1+e^θ}$。
具体代码如下：

In [73]:
p_congruent = 1 / (1 + np.exp(-trace.posterior["alpha"].mean())).to_pandas()
p_incongruent = 1 / (1 + np.exp(-(trace.posterior["beta"].mean()+trace.posterior["alpha"].mean()))).to_pandas()
print("alpha(一致条件) = ",p_congruent, "\n alpha+beta(不一致条件) = ", p_incongruent)

alpha(一致条件) =  0.8462798776599924 
 alpha+beta(不一致条件) =  0.8194362023007759


转换后可以发现，虽然一致条件的正确率略高于不一致条件，但这个差异并不具有统计学意义。

值得注意的是，模型预测的正确率与实际数据的正确率存在差异。
- 对于一致条件刺激，模型预测值为0.842，**低于**实际正确率为0.875。
- 对于不一致条件刺激，模型预测值为0.823，**高于**实际正确率为0.813。
- 模型预测两个条件的正确的差为(一致条件-不一致条件) = 0.02，而真实正确率的差异 = 0.06

可见，模型预测的效应更小。这是因为我们设置先验时，认为条件间的差异(即beta参数)服从均值为0，标准差为1的正态分布。

In [74]:
data.groupby('condition').correct.describe() 

,count,mean,std,min,25%,50%,75%,max
condition,,,,,,,,
0,48.0,0.8750,0.334219,0.0,1.0,1.0,1.0,1.0
1,48.0,0.8125,0.394443,0.0,1.0,1.0,1.0,1.0


假如我们存在先验知识，知道不一致条件的正确率低于一致条件，而不是他们可能相等。

我们可以设置一个有信息的先验，比如假设这个差异为 0.6，即 beta = 0.6。

需要注意的是，由于链接函数的存在，我们需要把 beta进行 logit转化，得到转化后的 beta 为 0.41。 

因此，我们假定beta的先验分布服从 均值为 0.4，标准差为0.2的正态分布。使得beta大部分值都大于0 (相对于之前得到的beta = -0.134 < 0)。

In [75]:
p = 0.6
beta = np.log(p/(1-p))
print("转化后的beta:", beta)

转化后的beta: 0.4054651081081642


In [76]:
with pm.Model() as GLM_model2:
    # 定义先验
    alpha = pm.Normal('alpha',mu=0,sd=1)
    beta = pm.Normal('beta',mu=0.4,sd=0.2) # 我们假定beta的先验分布服从 均值为 0.4，标准差为0.2的正态分布
    # x为自变量，是之前已经载入的数据
    x = pm.Data("x", data['condition'])
    # 线性模型：mu是确定性随机变量，这个变量的值完全由右端值确定
    z = alpha - beta * x                  # 这里使用减号是因为一致条件比不一致条件的正确率高          
    p = pm.Deterministic("p", pm.math.invlogit(z))  
    # Y的观测值，这是一个特殊的观测随机变量，表示模型数据的可能性。也可以表示模型的似然，通过 observed 参数来告诉这个变量其值是已经被观测到了的，不会被拟合算法改变
    y_obs = pm.Bernoulli("y_obs", p=p, observed=data["correct"])  
    trace3 = pm.sample(draws = 2000, tune=1000, target_accept=0.9,chains=2, cores= 2,return_inferencedata=True)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [beta, alpha]


Sampling 2 chains for 1_000 tune and 2_000 draw iterations (2_000 + 4_000 draws total) took 5 seconds.


In [77]:
p_congruent = 1 / (1 + np.exp(-trace3.posterior["alpha"].mean())).to_pandas()
p_incongruent = 1 / (1 + np.exp(-(trace3.posterior["beta"].mean()+trace3.posterior["alpha"].mean()))).to_pandas()
print("alpha(一致条件) = ", p_congruent, "\n alpha+beta(不一致条件) = ", p_incongruent)

alpha(一致条件) =  0.855171478001516 
 alpha+beta(不一致条件) =  0.8956504809430272


- 上一个模型模型预测两个条件的正确的差(一致条件-不一致条件) = 0.02，而真实正确率的差异 = 0.06。
- 该模型预测两个条件的正确的差 = 0.05。已经非常接近真实差异。

该结果说明了先验设置对于结果的影响。

感兴趣的同学可以在课后尝试不同的先验设置，并进行相应的的模型诊断、模型比较和统计推断的练习。